In [1]:
import shelve

## Code Shelf

In [2]:
shelf = shelve.open('CodeShelf')

## Add or Update

In [6]:
value = """
def display(display_list, cols=2, size=(8, 8)):
    rows = (len(display_list) + cols - 1)//cols
    fig, axes = plt.subplots(rows, cols, figsize=size)

    def get_axis(index):
        row, col = index//cols, index%cols
        return axes[row, col] if rows>1 else axes[col]
    
    for index, item in enumerate(display_list):
        image, label = item
        axis = get_axis(index)
        axis.imshow(tf.keras.utils.array_to_img(image))
        axis.set_xlabel(label)

items = []
for item in train_ds.take(2).as_numpy_iterator():
    items.append(item)

display(items, size=(2,2))
"""

key = 'display_image_dataset'

print(shelf.get(key))
shelf[key] = value
shelf.sync()


def display(display_list, cols=2, size=(8, 8)):
    rows = (len(display_list) + cols - 1)//cols
    fig, axes = plt.subplots(rows, cols, figsize=size)

    def get_axis(index):
        row, col = index//cols, index%cols
        return axes[row, col] if rows>1 else axes[col]
    
    for index, item in enumerate(display_list):
        image, label = item
        axis = get_axis(index)
        axis.imshow(tf.keras.utils.array_to_img(image))
        axis.set_xlabel(label)

items = []
for item in train_ds.take(2).as_numpy_iterator():
    items.append(item)

display(items, size=(2,2))



## Enumerate

In [3]:
keys = list(shelf.keys())
print(len(keys), keys)

17 ['plot_history', 'model_summary', 'run_summary', 'tf_imports', 'basic_image_augmentations', 'image_randaugment', 'run_summary_multioutput', 'show_dataset_samples', 'draw_bb_boxes', 'dense_boxes', 'save_frames_as_gif', 'display_gifs', 'display_gifs_ipython', 'openai_virtual_display', 'display_images_in_grid', 'plot_loss', 'color_distortion']


## Query

In [4]:
print(shelf['display_images_in_grid'])


def display(display_list, cols=2, size=(8, 8)):
    rows = (len(display_list) + cols - 1)//cols
    fig, axes = plt.subplots(rows, cols, figsize=size)

    def get_axis(index):
        row, col = index//cols, index%cols
        return axes[row, col] if rows>1 else axes[col]
    
    for index, image in enumerate(display_list):
        axis = get_axis(index)
        axis.imshow(tf.keras.utils.array_to_img(display_list[index]))

for image, mask in test_prep_ds.take(2):
    display([image, mask])

